<span style='font-size:x-large'>Case 1:</span><span style='font-size:large'> Assuming the match rate/charge dependency is linear</span>



In [2]:
# knowing match rates and Lyft's charge and assuming that their dependence is linear I define linear function equation for match rate

match_rate_1 = 0.93
match_rate_2 = 0.6

charge_1 = 3
charge_2 = 6

slope = (match_rate_2 - match_rate_1) / (charge_2 - charge_1)

y_intercept = match_rate_1 - slope * charge_1

def calculate_match_rate(charge_amount):
    return(slope * charge_amount + y_intercept)

print(calculate_match_rate(3))
print(calculate_match_rate(4.5))
print(calculate_match_rate(6))

0.9300000000000002
0.7650000000000001
0.6000000000000001


In [3]:
churn_when_match = 0.1
churn_when_no_match = 0.33

# define a function that calculates churn knowing the charge amount

def calculate_riders_month_churn(charge_amount):
    match_rate = calculate_match_rate(charge_amount)
    mismatch_rate = 1 - match_rate
    result = churn_when_match * match_rate + churn_when_no_match * mismatch_rate
    return(result)

print(calculate_riders_month_churn(6))
print(calculate_riders_month_churn(3))
print(calculate_riders_month_churn(4.5))

0.192
0.11609999999999998
0.15404999999999996


In [4]:
# now I will define a function that will allow us to calculate the net revenue per month knowing the charge amount and number of riders
# if match_rate = num_riders / num_rides, then num_rides = num_riders / match_rate

def calculate_revenue(num_riders, charge_amount):
    num_rides = num_riders / calculate_match_rate(charge_amount)
    result = num_rides * charge_amount
    return(result)

print(calculate_revenue(100, 6))
print(calculate_revenue(100, 3))
print(calculate_revenue(100, 4))

999.9999999999998
322.58064516129025
487.80487804878044


In [5]:
# and I will also define a function that allows us to calculate month-to-month riders retention within a cohort knowing the charge amount

def calculate_nth_month_rider_retention(initial_riders_amount, months, charge_amount):
    churn = calculate_riders_month_churn(charge_amount)
    riders_per_month = []
    riders_amount = initial_riders_amount
    i = 0
    while i <= months:
        nth_month_riders_amount = round(riders_amount - (initial_riders_amount * churn))
        if nth_month_riders_amount >= 0:
            riders_per_month.append(nth_month_riders_amount)
        else: 
            riders_per_month.append(0)
        riders_amount = nth_month_riders_amount
        i += 1
    result = riders_per_month[months-1]
    return(result)

# I also used this function to calculate retention for all 12 month simultaneously, thus - an array (: I know it's unoptimal though

print(calculate_nth_month_rider_retention(1000, 12, 6))
print(calculate_nth_month_rider_retention(1000, 12, 3))
print(calculate_nth_month_rider_retention(1000, 12, 4.5))

0
0
0


In [6]:
# here is a function that calculates net revenue per month using cohort size and charge amount

def calculate_net_revenue_to_month(initial_riders_amount, months, charge_amount):
    monthly_revenue = []
    i = 0
    while i < months: 
        riders_amount = calculate_nth_month_rider_retention(initial_riders_amount, i+1, charge_amount)
        rev = calculate_revenue(riders_amount, charge_amount)
        if rev >= 0:
            monthly_revenue.append(rev)
        else: 
            monthly_revenue.append(0)
        i += 1
    return(monthly_revenue)
           
print(sum(calculate_net_revenue_to_month(1000, 12, 3)))
print(sum(calculate_net_revenue_to_month(1000, 12, 6)))
print(sum(calculate_net_revenue_to_month(1000, 12, 4.5)))

12335.483870967739
21199.999999999996
16270.588235294115


In [7]:
# calculate roi of the cohort with selected charge amount

rider_cac = 20

def calculate_roi(riders_acquisited, charge_amount, months):
    roi = sum(calculate_net_revenue_to_month(riders_acquisited, months, charge_amount)) / (riders_acquisited * rider_cac)
    return(roi)

print(calculate_roi(1000, 3, 12))
print(calculate_roi(1000, 6, 12))
print(calculate_roi(1000, 4, 12))
print(calculate_roi(1000, 5.7, 12))

0.6167741935483869
1.0599999999999998
0.744390243902439
1.008530805687204


<span style='font-size:x-large'>Case 2:</span><span style='font-size:large'> Assuming</span> <span style='font-size:large'>the dependency is non\-linear</span>

Let's model this situation and find out if we really need to run the 3rd experiment



In [8]:
# let's imagine we conducted the second experiment and found out the dependency is nonlinear

match_rate_1 = 0.6
match_rate_2 = 0.93
match_rate_3 = 0.6 # I've also tried match rate == 0.93 here

charge_1 = 6
charge_2 = 3
charge_3 = 4.5

In [9]:
# I'll use polynomial interpolation to find out an equation behind the dependency

import numpy as np
from numpy.linalg import solve

points = [(charge_1, match_rate_1), (charge_2, match_rate_2), (charge_3, match_rate_3)]

x_matrix = np.array([[charge_1**2, charge_1, 1], [charge_2**2, charge_2, 1], [charge_3**2, charge_3, 1]])

y_vector = np.array([match_rate_1, match_rate_2, match_rate_3])

coefficients = solve(x_matrix, y_vector)

a, b, c = coefficients

print(coefficients)

# and define the new function to find out the match rate

def calculate_match_rate_2(charge_amount):
    return(a * charge_amount**2 + b * charge_amount + c)

print(calculate_match_rate_2(6))
print(calculate_match_rate_2(3))
print(calculate_match_rate_2(3.72)) #local max when match rate(4.5) = 0.93
print(calculate_match_rate_2(4.5))
print(calculate_match_rate_2(5.3)) #local min when match rate(4.5) = 0.6

[ 0.07333333 -0.77        2.58      ]
0.6000000000000001
0.9300000000000002
0.730416
0.6000000000000003
0.558933333333334


In [10]:
# and now we just duplicate all of the calculations

churn_when_match = 0.1
churn_when_no_match = 0.33

# define a function that allows us to calculate churn knowing the charge amount

def calculate_riders_month_churn_2(charge_amount):
    match_rate = calculate_match_rate_2(charge_amount)
    mismatch_rate = 1 - match_rate
    result = churn_when_match * match_rate + churn_when_no_match * mismatch_rate
    return(result)

print(calculate_riders_month_churn_2(6))
print(calculate_riders_month_churn_2(3))
print(calculate_riders_month_churn_2(4))
print(calculate_riders_month_churn_2(5.2))
print(calculate_riders_month_churn_2(3.72))

0.192
0.11609999999999998
0.17513333333333336
0.20144533333333342
0.16200432000000003


In [11]:
def calculate_revenue_2(num_riders, charge_amount):
    num_rides = num_riders / calculate_match_rate_2(charge_amount)
    result = num_rides * charge_amount
    return(result)

print(calculate_revenue_2(100, 6))
print(calculate_revenue_2(100, 3))
print(calculate_revenue_2(100, 4))

999.9999999999998
322.58064516129025
594.059405940594


In [12]:
def calculate_nth_month_rider_retention_2(initial_riders_amount, months, charge_amount):
    churn = calculate_riders_month_churn_2(charge_amount)
    riders_per_month = []
    riders_amount = initial_riders_amount
    i = 0
    while i <= months:
        nth_month_riders_amount = round(riders_amount - (initial_riders_amount * churn))
        riders_per_month.append(nth_month_riders_amount)
        riders_amount = nth_month_riders_amount
        i += 1
    result = riders_per_month[months-1]
    return(result)

print(calculate_nth_month_rider_retention_2(1000, 1, 6))
print(calculate_nth_month_rider_retention_2(1000, 1, 3))
print(calculate_nth_month_rider_retention_2(1000, 1, 4))

808
884
825


In [13]:
def calculate_net_revenue_to_month_2(initial_riders_amount, months, charge_amount):
    monthly_revenue = []
    i = 0
    while i < months: 
        riders_amount = calculate_nth_month_rider_retention_2(initial_riders_amount, i+1, charge_amount)
        rev = calculate_revenue_2(riders_amount, charge_amount)
        if rev >= 0:
            monthly_revenue.append(rev)
        else: 
            monthly_revenue.append(0)
        i += 1
    return(monthly_revenue)
           
print(sum(calculate_net_revenue_to_month_2(1000, 12, 3)))
print(sum(calculate_net_revenue_to_month_2(1000, 12, 6)))
print(sum(calculate_net_revenue_to_month_2(1000, 12, 5.2)))
print(sum(calculate_net_revenue_to_month_2(1000, 12, 3.72)))
print(sum(calculate_net_revenue_to_month_2(1000, 12, 4.5)))

12335.483870967739
21199.999999999996
18513.835877862606
13231.58309785109
15899.999999999993


In [14]:
rider_cac = 20

def calculate_roi_2(riders_acquisited, charge_amount, months):
    roi = sum(calculate_net_revenue_to_month_2(riders_acquisited, months, charge_amount)) / (riders_acquisited * rider_cac)
    return(roi)

print(calculate_roi_2(1000, 3, 12))
print(calculate_roi_2(1000, 6, 12))
print(calculate_roi_2(1000, 4, 12))
print(calculate_roi_2(1000, 5.3, 12))

0.6167741935483869
1.0599999999999998
0.7054455445544554
0.9434935591603043


<span style='font-size:x-large'>Case 3:</span><span style='font-size:large'> Assuming we stabilized retention at 10%</span>

<span style='font-size:small'>Trying to find even more optimal strategy</span>



In [15]:
retention_stabilized_at = 0.1

# redefining retention calculation

def calculate_nth_month_rider_retention_stabilized(initial_riders_amount, months, charge_amount):
    churn = calculate_riders_month_churn(charge_amount)
    riders_per_month = []
    riders_amount = initial_riders_amount
    i = 0
    while i <= months:
        nth_month_riders_amount = round(riders_amount - (initial_riders_amount * churn))
        if nth_month_riders_amount >= initial_riders_amount * retention_stabilized_at:
            riders_per_month.append(nth_month_riders_amount)
        else: 
            riders_per_month.append(initial_riders_amount * retention_stabilized_at)
        riders_amount = nth_month_riders_amount
        i += 1
    result = riders_per_month[months-1]
    return(result)

print(calculate_nth_month_rider_retention_stabilized(1000, 12, 6))
print(calculate_nth_month_rider_retention_stabilized(1000, 12, 3))
print(calculate_nth_month_rider_retention_stabilized(1000, 12, 4))

100.0
100.0
100.0


In [16]:
# as well as net revenue calculation function

def calculate_net_revenue_to_month_stabilized(initial_riders_amount, months, charge_amount):
    monthly_revenue = []
    i = 0
    while i < months: 
        riders_amount = calculate_nth_month_rider_retention_stabilized(initial_riders_amount, i+1, charge_amount)
        rev = calculate_revenue(riders_amount, charge_amount)
        if rev >= 0:
            monthly_revenue.append(rev)
        else: 
            monthly_revenue.append(0)
        i += 1
    return(monthly_revenue)
           
print(sum(calculate_net_revenue_to_month_stabilized(1000, 24, 3)))
print(sum(calculate_net_revenue_to_month_stabilized(1000, 24, 6)))
print(sum(calculate_net_revenue_to_month_stabilized(1000, 24, 5.2)))
print(sum(calculate_net_revenue_to_month_stabilized(1000, 24, 4.5)))

17587.096774193542
40799.99999999999
32651.16279069764
26999.999999999993


In [17]:
# and redefine roi

def calculate_roi_stabilized(riders_acquisited, charge_amount, months):
    roi = sum(calculate_net_revenue_to_month_stabilized(riders_acquisited, months, charge_amount)) / (riders_acquisited * rider_cac)
    return(roi)

print(calculate_roi_stabilized(1000, 3, 24))
print(calculate_roi_stabilized(1000, 6, 24))
print(calculate_roi_stabilized(1000, 4, 24))
print(calculate_roi_stabilized(1000, 5.3, 24))

0.8793548387096771
2.0399999999999996
1.1802439024390246
1.6792466765140337
